### 🖋 **Notebook Contents**

0. Initial Setup
1. Modelling
    - Benchmark Model
    - Hyperparameter Tuning
    - Final Model
2. Cluster Analysis
3. Conclusion
4. Recommendation

****

## `Initial Setup`

In [43]:
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, RobustScaler, StandardScaler
import category_encoders as ce

# mlflow and dagshub tracking
import mlflow
import dagshub

# get experiment_id
sys.path.append(os.path.abspath("../src/"))
from get_or_create_mlflow_experiments import get_experiment_id
EXP_ID = get_experiment_id('Credit_Card_Clustering') # experiment id

# model
from sklearn.cluster import KMeans, MeanShift # centroid-based
from sklearn.cluster import DBSCAN, OPTICS # density-based
from sklearn.cluster import AgglomerativeClustering # hierarchical-based
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import Birch
from hdbscan import HDBSCAN # hierarchical density-based

# metric evaluation
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

import warnings
warnings.filterwarnings('ignore')

In [44]:
# dagshub and mlflow setup
DAGSHUB_USERNAME = 'fnkhairudin'
DAGSHUB_EMAIL = 'fnkhairudin@gmail.com'
DAGSHUB_REPO = 'Credit-Card-Clustering-Customer-Segmentation'
DAGSHUB_TOKEN = 'b1b10b82350be145d4ea4d0d14d8514de68ae6cf' # dagshub.auth.get_token()
DASGHUB_URI = f'https://dagshub.com/{DAGSHUB_USERNAME}/{DAGSHUB_REPO}.mlflow'

os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN
os.environ['MLFLOW_TRACKING_URI'] = DASGHUB_URI

In [45]:
data = pd.read_csv("../data/processed/clean_cc_data.csv")
data.sample(10)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
7294,9.048773,0.909091,316.20,0.00,316.20,0.000000,1.000000,0.000000,1.000000,0.0,0.0,24.0,2000.0,391.398298,89.505939,0.600000,12.0
4007,231.089583,1.000000,1547.92,1547.92,0.00,0.000000,1.000000,1.000000,0.000000,0.0,0.0,12.0,9000.0,1230.772375,196.595495,0.333333,12.0
1415,920.922411,1.000000,1227.93,459.55,768.38,0.000000,0.916667,0.166667,0.916667,0.0,0.0,15.0,8500.0,2219.823750,224.328091,0.000000,12.0
5922,92.239093,0.909091,1525.10,0.00,1525.10,0.000000,0.833333,0.000000,0.833333,0.0,0.0,13.0,4000.0,1682.312643,157.835032,1.000000,12.0
5166,1494.696950,0.666667,0.00,0.00,0.00,3020.521475,0.000000,0.000000,0.000000,0.5,20.0,0.0,3000.0,582.828406,318.949630,0.000000,6.0
44,5315.945594,1.000000,1525.44,469.80,1055.64,0.000000,1.000000,0.333333,1.000000,0.0,0.0,20.0,8500.0,1430.341501,1318.714920,0.000000,12.0
717,1024.761905,1.000000,77.99,77.99,0.00,0.000000,0.166667,0.166667,0.000000,0.0,0.0,2.0,1200.0,555.233730,303.743033,0.000000,12.0
7972,22.161656,0.636364,1240.58,0.00,1240.58,0.000000,0.666667,0.000000,0.666667,0.0,0.0,15.0,1500.0,1817.791324,107.871575,0.500000,12.0
1488,937.434519,1.000000,1124.88,1085.63,39.25,0.000000,0.416667,0.333333,0.083333,0.0,0.0,9.0,2500.0,993.196174,252.572491,0.000000,12.0
1290,2006.218006,1.000000,4572.63,3484.50,1088.13,0.000000,0.500000,0.416667,0.250000,0.0,0.0,28.0,6500.0,1670.202477,1842.365907,0.000000,12.0


In [46]:
print(f'Jumlah baris dan kolom: {data.shape[0]} baris dan {data.shape[1]} kolom')

# skimming dataset
pd.DataFrame(
                {
                'kolom': data.columns.values,
                'data_type': data.dtypes.values,
                'null_value(%)': data.isna().mean().values * 100,
                'n_unique': data.nunique().values,
                'zero_value' : [True if (data[col] == 0).any() else False for col in data.columns],
                'neg_value' : [True if (data[col].dtype == int or data[col].dtype == float) and (data[col] < 0).any() else False for col in data.columns],
                'min': data.min().values,
                'max': data.max().values,
                'sample_unique': [data[col].unique() for col in data.columns]
                }
            )

Jumlah baris dan kolom: 8950 baris dan 17 kolom


,kolom,data_type,null_value(%),n_unique,zero_value,neg_value,min,max,sample_unique
0,BALANCE,float64,0.0,8871,True,False,0.000000,19043.13856,"[40.900749, 3202.467416, 2495.148862, 1666.670..."
1,BALANCE_FREQUENCY,float64,0.0,43,True,False,0.000000,1.00000,"[0.818182, 0.909091, 1.0, 0.636364, 0.545455, ..."
2,PURCHASES,float64,0.0,6203,True,False,0.000000,49039.57000,"[95.4, 0.0, 773.17, 1499.0, 16.0, 1333.28, 709..."
3,ONEOFF_PURCHASES,float64,0.0,4014,True,False,0.000000,40761.25000,"[0.0, 773.17, 1499.0, 16.0, 6402.63, 661.49, 1..."
4,INSTALLMENTS_PURCHASES,float64,0.0,4452,True,False,0.000000,22500.00000,"[95.4, 0.0, 1333.28, 688.38, 436.2, 200.0, 920..."
5,CASH_ADVANCE,float64,0.0,4323,True,False,0.000000,47137.21176,"[0.0, 6442.945483, 205.788017, 346.81139, 2301..."
6,PURCHASES_FREQUENCY,float64,0.0,47,True,False,0.000000,1.00000,"[0.166667, 0.0, 1.0, 0.083333, 0.666667, 0.333..."
7,ONEOFF_PURCHASES_FREQUENCY,float64,0.0,47,True,False,0.000000,1.00000,"[0.0, 1.0, 0.083333, 0.166667, 0.25, 0.916667,..."
8,PURCHASES_INSTALLMENTS_FREQUENCY,float64,0.0,47,True,False,0.000000,1.00000,"[0.083333, 0.0, 0.583333, 1.0, 0.25, 0.916667,..."
9,CASH_ADVANCE_FREQUENCY,float64,0.0,47,True,False,0.000000,1.00000,"[0.0, 0.25, 0.083333, 0.166667, 0.333333, 0.36..."


## `Feature Engineering (Experiment)`

## `Modelling`

In [47]:
# dataset untuk modelling
data_modelling = data.copy()

In [53]:
# fungsi untuk melakukan model training
def log_training_model(est, model_name:str, score_result:list, dataset:pd.DataFrame, name_experiment:str):
    """
    est : Estimator object
    model_name : Estimator name
    score_result : Empty list to store the metric result
    dataset : Dataset for training the estimator
    name_experiment : experiment name
    """
    # model params
    model_params = est.get_params()
    # set_tag
    tags = {"model_name": model_name, "experiment-n": name_experiment}
    
    # tag and parameters mlflow ui
    mlflow.set_tags(tags)
    mlflow.log_params(model_params)   
    
    print(f"training model {model_name} ......")
    # hasil cluster untuk masing-masing point data
    result = est.fit_predict(dataset)

    # jika terbentuk HANYA 1 cluster --> metric evaluation tidak bisa melakukan perhitungan
    if len(np.unique(result)) == 1:
        score_result.append({
        'Model' : model_name,
        'silhouette_score': np.nan,
        'calinski_harabasz_score': np.nan,
        'davies_bouldin_score': np.nan
        })
        # log metrics pada mlflow ui
        mlflow.log_metrics({
                'silhouette': np.float('nan'),
                'calinski_harabasz': np.float('nan'),
                'davies_bouldin': np.float('nan')
                })
        # log model pada mlflow ui
        mlflow.sklearn.log_model(est, "sk_model")

    # jika terbentuk lebih dari 1 cluster
    else:
        # hasil clustering
        score_result.append({
        'Model' : model_name,
        'silhouette_score': silhouette_score(dataset, result),
        'calinski_harabasz_score': calinski_harabasz_score(dataset, result),
        'davies_bouldin_score': davies_bouldin_score(dataset, result)
        })
        # log metrics pada mlflow ui
        mlflow.log_metrics({
                        'silhouette': silhouette_score(dataset, result),
                        'calinski_harabasz': calinski_harabasz_score(dataset, result),
                        'davies_bouldin': davies_bouldin_score(dataset, result)
                            })
        # log model pada mlflow ui
        mlflow.sklearn.log_model(est, "sk_model")
    print(f"training model {model_name} is DONE.")

In [54]:
# model
kmeans = KMeans(n_clusters=5)
meanshift = MeanShift()
dbscan = DBSCAN()
optics = OPTICS()
agglomerative = AgglomerativeClustering()
# affinity = AffinityPropagation()
birch = Birch()
hdbscan = HDBSCAN()

# collect model
model = {
    'KMeans': kmeans,
    'MeanShift': meanshift,
    'DBSCAN': dbscan,
    'OPTICS': optics,
    'AgglomerativeClustering': agglomerative,
    # 'AffinityPropagation': affinity,
    'Birch' : birch,
    'HDBSCAN' : hdbscan
}

# hasil metrik evaluasi
metric_result = []

# melakukan set up the mlflow uri untuk tracking hasil eksperiment
mlflow.set_tracking_uri(DASGHUB_URI)

# melakukan tracking eksperiment menggunakan mlflow
for name, estimator in model.items(): 
    with mlflow.start_run(experiment_id=EXP_ID):
        # call fungsi untuk training model
        log_training_model(est=estimator, 
                           model_name=name, 
                           score_result=metric_result, 
                           dataset=data_modelling, 
                           name_experiment='first init experiment'
                           )

training model KMeans ......
training model KMeans is DONE.
training model MeanShift ......
training model MeanShift is DONE.
training model DBSCAN ......


2023/10/11 11:43:57 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


training model DBSCAN is DONE.
training model OPTICS ......


2023/10/11 11:44:31 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


training model OPTICS is DONE.
training model AgglomerativeClustering ......


2023/10/11 11:44:56 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


training model AgglomerativeClustering is DONE.
training model Birch ......
training model Birch is DONE.
training model HDBSCAN ......


2023/10/11 11:45:59 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


training model HDBSCAN is DONE.


In [55]:
# CSS properties
PROPS = 'color:#080808;font-weight:bold;background-color:#85bb65'

# tampilkan hasil training model dalam bentuk DataFrame
metric_result_df = pd.DataFrame(metric_result).style.\
    highlight_min(subset='davies_bouldin_score', props=PROPS).\
    highlight_max(subset=['silhouette_score', 'calinski_harabasz_score'], props=PROPS)
metric_result_df

,Model,silhouette_score,calinski_harabasz_score,davies_bouldin_score
0,KMeans,0.379409,2585.059481,1.306491
1,MeanShift,0.426253,152.296202,0.608292
2,DBSCAN,nan,nan,nan
3,OPTICS,-0.559804,6.780031,1.384889
4,AgglomerativeClustering,0.392781,3048.942835,1.271941
5,Birch,0.396881,2537.275301,1.011750
6,HDBSCAN,-0.444763,15.338152,1.430466
